In [2]:
!pip install peft
!pip install modelscope

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.8/283.8 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.1/443.1 kB 27.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 91.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 13.2 MB/s eta 0:00:00
  Created wheel for oss2: filename=oss2-2.18.6-py3-none-any.whl size=118354 sha256=e80409e339c36d9181d81bd7a879d18e8639f7b8e1884e9a9eb6c62b8bbbe118
  Stored in directory: /root/.cache/pip/wheels/e9/1c/df/6256a3d22097f6e1a30edd892de172054fd27875e0a349b4a4
  Created wheel for 

In [3]:
#查看GPU信息
import subprocess
# 执行nvidia-smi命令
result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
# 打印输出
print(result.stdout)

Mon Jun 17 04:33:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0              26W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 【数据处理】

In [ ]:
# import datasets
# dataset = datasets.load_dataset("yelp_review_full")

In [4]:
import modelscope
from modelscope.msdatasets import MsDataset
#【下载数据集】
HC3=MsDataset.load('simpleai/HC3-Chinese', subset_name='baike', split='train') #调用HC3数据集
dataset=HC3.to_hf_dataset() #将MsDataset转换成huggingface dataset格式，方便后续处理
print("【数据集下载完成】")

2024-06-17 04:33:07,896 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-06-17 04:33:07,902 - modelscope - INFO - TensorFlow version 2.15.0 Found.
2024-06-17 04:33:07,903 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-06-17 04:33:07,903 - modelscope - INFO - No valid ast index found from /root/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-06-17 04:33:07,985 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 cf8dafbb4e308ed48127fabe0ef1ceec and a total number of 980 components indexed
/opt/conda/lib/python3.10/site-packages/datasets/load.py:2524: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/datasets/load.py:926: FutureWarning: The repository for HC3-Chinese contains 

Generating train split: 0 examples [00:00, ? examples/s]

【数据集下载完成】


In [5]:
from datasets import Dataset
#【调整数据集格式】
def data_init(dataset):
    ds=[]
    cnt=dataset.num_rows
    for i in range(cnt):
        example=dataset[i]
        ds.append({"label":0,"text":example["human_answers"][0]})
        ds.append({"label":1,"text":example["chatgpt_answers"][0]})
    return Dataset.from_list(ds)

dataset=data_init(dataset) # 调整数据集内容
print(dataset)
dataset=dataset.shuffle(seed=233).select(range(5000)) #随机选一部分

#数据集划分 train:val:test=8:1:1
data_=dataset.train_test_split(train_size=0.8,seed=233) #数据集划分
data_train=data_["train"]
data__=data_["test"].train_test_split(train_size=0.5,seed=233)
data_val=data__["train"]
data_test=data__["test"]

print("【data_train】",data_train)
print("【data_val】",data_val)
print("【data_test】",data_test)

Dataset({
    features: ['label', 'text'],
    num_rows: 9234
})
【data_train】 Dataset({
    features: ['label', 'text'],
    num_rows: 4000
})
【data_val】 Dataset({
    features: ['label', 'text'],
    num_rows: 500
})
【data_test】 Dataset({
    features: ['label', 'text'],
    num_rows: 500
})


## 【模型】

In [6]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments,Trainer,DataCollatorWithPadding

#【加载分词器】
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B") #("bert-base-cased")
tokenizer.pad_token_id = tokenizer.eos_token_id #Qwen特性，需要指定一下pad_token_id
print(tokenizer)
# print("【pad_token_id】",tokenizer.pad_token_id)

def tokenize_function(examples):
    return tokenizer(examples["text"],padding="max_length",truncation=True,max_length=512)

token_train=data_train.map(tokenize_function, batched=True)
token_val=data_val.map(tokenize_function, batched=True)
print("【token_train[0]】",token_train[0])

train_dataset = token_train
eval_dataset = token_val

2024-06-17 04:33:40.508356: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 04:33:40.508488: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 04:33:40.630987: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2TokenizerFast(name_or_path='Qwen/Qwen1.5-0.5B', vocab_size=151643, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

【token_train[0]】 {'label': 0, 'text': '网络节点是指一台电脑或其他设备与一个有独立地址和具有传送或接收数据功能的网络相连。节点可以是工作站、客户、网络用户或个人计算机，还可以是服务器、打印机和其他网络连接的设备。每一个工作站﹑服务器、终端设备、网络设备，即拥有自己唯一网络地址的设备都是网络节点。整个网络就是由这许许多多的网络节点组成的，把许多的网络节点用通信线路连接起来，形成一定的几何关系，这就是计算机网络拓扑。', 'input_ids': [71356, 92374, 104442, 106621, 104145, 105994, 101044, 57218, 46944, 18830, 102024, 46477, 33108, 100629, 112523, 57191, 106585, 20074, 98380, 9370, 71356, 111060, 1773, 92374, 73670, 20412, 114896, 5373, 100017, 5373, 71356, 20002, 57191, 99605, 104564, 3837, 104468, 20412, 89047, 5373, 117648, 105504, 71356, 64064, 9370, 101044, 1773, 104367, 114896, 123930, 239, 89047, 5373, 104992, 101044, 5373, 71356, 101044, 3837, 91676, 103926, 99283, 102157, 71356, 46477, 9370, 101044, 100132, 71356, 92374, 1773, 101908, 71356, 99486, 67071, 43288, 99454, 100694, 42140, 9370, 71356, 92374, 107339, 3837, 99360, 100694, 9370, 71356, 92374, 11622, 104516, 104634, 64064, 99793, 3837, 101894, 102495, 111867, 100145, 3837, 104301, 104564, 71356, 100786, 102498, 

In [ ]:
#使用BERT模型
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased",num_labels=5)
# print(model)

In [7]:
#【加载模型】
id2label = {0: "human", 1: "chatgpt"}
label2id = {"human": 0, "chatgpt": 1}
#使用Qwen1.5模型
model = AutoModelForSequenceClassification.from_pretrained("Qwen/Qwen1.5-0.5B",num_labels=2,id2label=id2label,label2id=label2id)
model.config.pad_token_id=model.config.eos_token_id #这里也要指定一下pad_token_id，不然训练时会报错 "ValueError: Cannot handle batch sizes > 1 if no padding token is defined."
print("【model】\n",model)
print("【model.config】\n",model.config)
print("【model.config.pad_token_id】",model.config.pad_token_id)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen1.5-0.5B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


【model】
 Qwen2ForSequenceClassification(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1024, out_features=2816, bias=False)
          (up_proj): Linear(in_features=1024, out_features=2816, bias=False)
          (down_proj): Linear(in_features=2816, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm(

## 【训练】

In [8]:
#【训练参数】
from datasets import load_metric
import numpy as np

training_args = TrainingArguments(
    output_dir="pt_save_pretrained",
    evaluation_strategy="epoch", #每跑完一个epoch输出一下测试信息
    num_train_epochs=2,
    per_device_train_batch_size=4, # 一共要跑 len(dataset)/batch_size * epoch 个step
                                  # [模型=Qwen1.5-0.5B, batch_size=4]：完全微调显存13.3GB，LoRA微调显存8.7GB
#     gradient_accumulation_steps=2,
    #     load_best_model_at_end=True,
    save_strategy="no",  #关闭自动保存模型（Kaggle上磁盘空间不太够）
#     save_total_limit=1, #保存检查点数量的限制
)

metric=load_metric('accuracy') #评估指标

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def get_trainer(model): 
    return  Trainer( 
        model=model, 
        args=training_args, 
        tokenizer=tokenizer,
        train_dataset=train_dataset, 
        eval_dataset=eval_dataset, 
        compute_metrics=compute_metrics,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding=True,return_tensors="pt"), #给数据添加padding弄成batch
    ) 

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_34/3640500368.py:17: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric=load_metric('accuracy') #评估指标
/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release o

In [ ]:
#【完全微调】
print("【开始训练】")
trainer=get_trainer(model)
trainer.train()

tokenizer.save_pretrained("./full_model_tokenizer") 
model.save_pretrained("./full_model")

#Kaggle注意：
#每次训练之后restart以释放显存！
#factory也reset一下，不然磁盘空间会爆！

In [9]:
#【PEFT-LoRA微调】
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    task_type="SEQ_CLS", #任务类型：分类 
    target_modules=["q_proh","k_proj","v_proj","o_proj"],  # 这个不同的模型需要设置不同的参数，主要看模型中的attention层
    inference_mode=False, # 关闭推理模式 (即开启训练模式)
    r=8, # Lora 秩
    lora_alpha=16, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1 # Dropout 比例
)

peft_model = get_peft_model(model, peft_config) # 加载lora参数peft框架

print('PEFT参数量：') 
peft_model.print_trainable_parameters() 

print("【开始训练】")
peft_trainer=get_trainer(peft_model)
peft_trainer.train()

tokenizer.save_pretrained("./peft_model_tokenizer") 
peft_model.save_pretrained("./peft_model")

PEFT参数量：
trainable params: 1,181,696 || all params: 465,171,456 || trainable%: 0.2540
【开始训练】


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.102500,0.113912,0.986000
2,0.052600,0.107569,0.986000


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## 【测试】

In [10]:
import torch
from transformers import DataCollatorWithPadding,AutoTokenizer,AutoModelForSequenceClassification

def classify(example,show): #对example进行预测
    text=example["text"]
    label=example["label"]
#     print("【example】",example)
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt").to('cuda')
#     print("【inputs】",inputs)
    with torch.no_grad(): 
        output = inference_model(**inputs) 
        pred = output.logits.argmax(dim=-1).item() 
    if show:
        print("【预测{}!】Label: {}, Pred_Label: {}\nText: {}".format("正确" if label==pred else "错误",id2label[label],id2label[pred],text))
    else:
        return pred,label

# inference_model=model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained("./peft_model_tokenizer")
inference_model = AutoModelForSequenceClassification.from_pretrained("./peft_model").to('cuda') #读取训练好的模型
print("【model】\n",inference_model)
print("【model.config】\n",inference_model.config)
print("【model.config.pad_token_id】",inference_model.config.pad_token_id)
data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding=True,return_tensors="pt")

id2label = {0: "human", 1: "chatgpt"}
label2id = {"human": 0, "chatgpt": 1}

classify(data_test[0],1) #随便测试一个数据

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen1.5-0.5B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


【model】
 Qwen2ForSequenceClassification(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=1024, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=1024, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
            (lo

In [11]:
from datasets import load_metric
from tqdm import tqdm 
metric=load_metric('accuracy')

print("【测试集】",data_test)
inference_model.eval() 
for i,example in enumerate(tqdm(data_test)): 
    pred, label = classify(example,0)
#     print("NO.{} 预测{}! Label: {}, Pred_Label: {}".format(i,"正确" if label==pred else "错误",id2label[label],id2label[pred]))
    metric.add(predictions=pred, references=label)
print(metric.compute()) 

/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


【测试集】 Dataset({
    features: ['label', 'text'],
    num_rows: 500
})


100%|██████████| 500/500 [00:17<00:00, 29.07it/s]

{'accuracy': 0.982}
